## Python loading

In [321]:
import sys

import pandas as pd
from pandas import DataFrame, Series

import numpy as np

import matplotlib.pyplot as plt
import matplotlib

#import xgboost as xgb

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier 

%matplotlib inline

In [322]:
train = pd.read_csv("../../data/train.csv", index_col=0)
test = pd.read_csv("../../data/test.csv", index_col=0)

train.head(2)

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby


## Data fitting

In [323]:
train.SexuponOutcome.value_counts()

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
Name: SexuponOutcome, dtype: int64

In [328]:
train = pd.read_csv("../../data/train.csv", index_col=0)
test = pd.read_csv("../../data/test.csv", index_col=0)

def categorising(dataset):
    opt = dataset.copy()

    opt["Intact"] = opt.SexuponOutcome.replace("Intact Male", "Yes")
    opt.Intact = opt.Intact.replace("Neutered Male", "No")
    opt.Intact = opt.Intact.replace("Spayed Female", "No")
    opt.Intact = opt.Intact.replace("Intact Female", "Yes")
    opt.Intact = opt.Intact.replace("Unknown", np.nan)

    opt.Intact = opt.Intact.astype("category")
    opt["IntactValue"] = opt.Intact.cat.codes

    opt.SexuponOutcome = opt.SexuponOutcome.replace("Neutered Male", "Male")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Male", "Male")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Spayed Female", "Female")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Female", "Female")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Unknown", np.nan)

    opt.SexuponOutcome = opt.SexuponOutcome.astype("category")
    opt["SexuponOutcomeValue"] = opt.SexuponOutcome.cat.codes

    return opt

train = categorising(train)
test = categorising(test)

opt = train.copy()

opt.Intact.value_counts()

No     18599
Yes     7036
dtype: int64

In [240]:


opt.Color.value_counts()

opt[(opt.Color == "Blue") & (opt.AnimalType == "Dog")].Breed.value_counts()

Pit Bull Mix                     34
Weimaraner Mix                    4
Labrador Retriever Mix            3
Blue Lacy Mix                     3
Weimaraner                        3
Chihuahua Shorthair Mix           2
Pit Bull                          2
Cane Corso Mix                    1
Australian Kelpie/Blue Lacy       1
Chow Chow                         1
Great Dane Mix                    1
Staffordshire Mix                 1
Australian Kelpie Mix             1
Great Dane/Pit Bull               1
Chinese Sharpei/Great Dane        1
Chinese Sharpei                   1
Pit Bull/Flat Coat Retriever      1
Australian Shepherd/Chow Chow     1
Pit Bull/Weimaraner               1
Blue Lacy/Australian Kelpie       1
Standard Schnauzer Mix            1
Name: Breed, dtype: int64

In [241]:
opt.AnimalType = opt.AnimalType.astype("category")
opt.AnimalType.value_counts()

Dog    15595
Cat    11134
dtype: int64

In [284]:
opt.OutcomeType = opt.OutcomeType.astype("category")
opt.OutcomeType.value_counts()


Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
dtype: int64

## Prediction

In [285]:
#create and train the random forest
    #multi-core CPUs can use: rf = RandomForestClassifier(n_estimators=100, n_jobs=2)
rf = RandomForestClassifier(n_estimators=100, n_jobs=2)

d_train = opt[["SexuponOutcomeValue", "IntactValue"]].values

#d_train
d_target = opt.OutcomeType.cat.codes.values

In [248]:
d_train
#d_train.reshape(-1, 1)

[array([1, 0, 1, ..., 1, 1, 1], dtype=int8),
 array([1, 0, 1, ..., 1, 1, 1], dtype=int8)]

In [288]:
rf.fit(d_train, d_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [342]:
for index, row in test.head(50).iterrows():
    print([row['SexuponOutcomeValue'], row['IntactValue']])
    print(rf.predict([[row['SexuponOutcomeValue'], row['IntactValue']]]))

[0, 1]
[0]
[0, 0]
[0]
[1, 0]
[0]
[1, 1]
[0]
[1, 0]
[0]
[1, 0]
[0]
[1, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 1]
[0]
[0, 0]
[0]
[1, 0]
[0]
[1, 0]
[0]
[0, 0]
[0]
[1, 0]
[0]
[1, 0]
[0]
[0, 0]
[0]
[0, 1]
[0]
[0, 0]
[0]
[-1, -1]
[4]
[1, 0]
[0]
[0, 1]
[0]
[0, 0]
[0]
[1, 0]
[0]
[0, 0]
[0]
[1, 1]
[0]
[1, 0]
[0]
[1, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[1, 0]
[0]
[1, 0]
[0]
[0, 1]
[0]
[1, 0]
[0]
[1, 0]
[0]
[1, 0]
[0]
[0, 0]
[0]
[1, 0]
[0]
[0, 1]
[0]
[1, 1]
[0]
[0, 0]
[0]
[1, 1]
[0]
[1, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[1, 0]
[0]
[0, 0]
[0]
[1, 1]
[0]
